#IPyParallel example notebook
This notebok presents the application of ipyparallel for parallel computing. For the example dummy functions are used. Any processing can be parallelized on the basis of this notebook.


## Imports

In [ ]:
import numpy as np
from ipyparallel import Client
import time as t

## Create list of clients for parallel use, properties ids contains id's of these Clients

In [ ]:
cList = Client() 

## Prepare functions for processing
Input parameters can not be modified thus its copy is created.

In [ ]:
def functionToDO(data):
    data = data.copy()

    for i in range(len(data)):
        data[i] = data[i] ** 3
    return data


def functionSaveData(data, nameOfFile, potega):
    file = open(nameOfFile, 'w')
    for item in data:
        res = 1;
        licznik = item * potega
        while licznik > 0:
            res = res * licznik
            licznik = licznik - 1

        file.write("%s\n" % res)
    file.close()
    return 'ok'


def functionSaveData2(nameOfFile):
    file = open(nameOfFile, 'w')

    licznik = 100
    res = 1;
    while licznik > 0:
        res = res * licznik
        licznik = licznik - 1
        file.write("%s\n" % res)
    file.close()
    return 'ok'

## Prepare dummy dataset

In [ ]:
dataSet=[]
for i in range(1,35):
    dataSet.append(i)

## Start the engine 

In [96]:
result=cList[0].apply_sync(functionToDO, dataSet)

## Save to file using single engine

In [97]:
cList[0].apply_sync(functionSaveData, dataSet,'result.txt',3)

'ok'

## Prepare data for processing

In [98]:
number_of_tasks=300
_data=[]
_powers=[]
_filenames=[]
for i in range(1, number_of_tasks + 1):
    _data.append(dataSet)
    _powers.append(i)
    _filenames.append('result_' + str(i) + '.txt')


## Save to file by multiple engines without waiting for execution end, no load balanced

In [99]:
lview=cList[:]
results1=parallel_result = lview.map(functionSaveData, _data, _filenames, _powers)    

## Save to file by multiple engines without waiting for execution end, with load balanced

In [100]:
lview = cList.load_balanced_view()
result1=parallel_result = lview.map(functionSaveData, _data, _filenames, _powers)

## Another dummy function

In [ ]:
def wait(times):
    import time
    tic = time.time()
    time.sleep(times)
    return time.time()-tic

## Synchronous function call

In [126]:
dview = cList[:] 
dview.block=True
dview['a'] = 5
dview['b'] = 10
dview['data']=dataSet

delays=[1, 2, 3, 4]

ar = dview.apply_async(wait, 1)
ar.get()

[1.0004768371582031,
 1.0000030994415283,
 1.0009777545928955,
 1.0004770755767822]

## Import numpy here and everywhere

In [128]:
with cList[:].sync_imports():
    import numpy

importing numpy on engine(s)


## Example of using %px cell magic
The %px magic executes a single Python command on the engines specified by the targets attribute of the DirectView instance

In [134]:
%px a = numpy.random.rand(2,2)


In [136]:
%px numpy.linalg.eigvals(a)


Out[0:8]: array([ 0.43618706,  0.72317614])

Out[1:8]: array([ 0.84667598,  0.16517367])

Out[2:8]: array([ 0.75934655,  0.06421126])

Out[3:8]: array([-0.33629903,  0.91484145])

In [140]:
%px print ('hi')

[stdout:0] hi
[stdout:1] hi
[stdout:2] hi
[stdout:3] hi


In [142]:
%px %pylab inline

[stdout:0] Populating the interactive namespace from numpy and matplotlib
[stdout:1] Populating the interactive namespace from numpy and matplotlib
[stdout:2] Populating the interactive namespace from numpy and matplotlib
[stdout:3] Populating the interactive namespace from numpy and matplotlib


In [ ]:
%px plot(rand(100))